In [1]:
import pandas as pd

# Load Excel data
df_running_order = pd.read_excel('Running Order Table Pocono Cup 2025.xlsx', sheet_name='MotorSports')

# Clean data
df_running_order = df_running_order.drop(index=0).reset_index(drop=True)
df_running_order.rename(columns={'Unnamed: 0': 'Lap'}, inplace=True)

# Define caution periods and restart laps
caution_periods = [(0, 7), (32, 35), (43, 48), (56, 60), (78, 80), (84, 87), (97, 100), (126, 130)]
restart_laps = [end + 1 for _, end in caution_periods]

results = []

# Loop over restart laps
for lap in restart_laps:
    prev_lap = lap - 1
    
    if prev_lap not in df_running_order['Lap'].values:
        continue

    running_order_row = df_running_order[df_running_order['Lap'] == prev_lap].iloc[0]
    running_order = running_order_row[list(range(1, 41))].dropna().astype(int).tolist()

    restart_row = df_running_order[df_running_order['Lap'] == lap].iloc[0]
    top_16_drivers = restart_row[list(range(1, 41))].dropna().astype(int).head(16).tolist()

    for driver in top_16_drivers:
        position_before = running_order.index(driver) + 1 if driver in running_order else None

        results.append({
            'Lap': lap,
            'Driver': driver,
            'Position_Before_Choose': position_before
        })

# Convert to DataFrame
results_df = pd.DataFrame(results)

# Show the DataFrame
print(results_df)

     Lap  Driver  Position_Before_Choose
0      8      11                       1
1      8      17                       2
2      8      77                       3
3      8      45                       4
4      8      19                       5
..   ...     ...                     ...
123  131      54                      19
124  131      99                      10
125  131      20                      12
126  131      77                      15
127  131      43                      16

[128 rows x 3 columns]


In [2]:
import pandas as pd

# Load running order data
df_running_order = pd.read_excel('Running Order Table Pocono Cup 2025.xlsx', sheet_name='MotorSports')
xls_distance = pd.ExcelFile('Pocono Choose Line Export - Reorganized.xlsx')

# Clean running order
df_running_order = df_running_order.drop(index=0).reset_index(drop=True)
df_running_order.rename(columns={'Unnamed: 0': 'Lap'}, inplace=True)

# Define restart laps
restart_laps = [8, 36, 49, 61, 81, 88, 101, 131]
car_sheets = xls_distance.sheet_names

results = []

for lap in restart_laps:
    prev_lap = lap - 1
    if prev_lap not in df_running_order['Lap'].values:
        continue

    running_order_row = df_running_order[df_running_order['Lap'] == prev_lap].iloc[0]
    running_order = running_order_row[list(range(1, 41))].dropna().astype(int).tolist()

    restart_row = df_running_order[df_running_order['Lap'] == lap].iloc[0]
    top_16_drivers = restart_row[list(range(1, 41))].dropna().astype(int).head(16).tolist()

    for driver in top_16_drivers:
        position_before = running_order.index(driver) + 1 if driver in running_order else None

        choose_distance = None
        lane_choice = 'Outside'  # Default if missing

        sheet_name = f'Car {driver}'
        if sheet_name in car_sheets:
            df_car = pd.read_excel(xls_distance, sheet_name=sheet_name)
            if 'Restart Choose (ft)' in df_car.columns and 'Lap' in df_car.columns:
                lap_row = df_car[df_car['Lap'] == lap]
                if not lap_row.empty:
                    choose_distance = lap_row['Restart Choose (ft)'].values[0]
                    if pd.notna(choose_distance):
                        if 0 <= choose_distance <= 7:
                            lane_choice = 'Outside'
                        elif 8 <= choose_distance <= 40:
                            lane_choice = 'Inside'
                        else:
                            lane_choice = 'Outside'

        results.append({
            'Restart Lap': lap,
            'Driver (Car Number)': driver,
            'Position Before Choose': position_before,
            'Choose Distance (ft)': choose_distance,
            'Lane Choice': lane_choice
        })

# Convert to DataFrame
df_results = pd.DataFrame(results)

# Save to Excel if needed
df_results.to_excel('Pocono_Restart_Lane_Choice.xlsx', index=False)

# Print preview
print(df_results)


     Restart Lap  Driver (Car Number)  Position Before Choose  \
0              8                   11                       1   
1              8                   17                       2   
2              8                   77                       3   
3              8                   45                       4   
4              8                   19                       5   
..           ...                  ...                     ...   
123          131                   54                      19   
124          131                   99                      10   
125          131                   20                      12   
126          131                   77                      15   
127          131                   43                      16   

    Choose Distance (ft) Lane Choice  
0                   None     Outside  
1                   None     Outside  
2                   None     Outside  
3                   None     Outside  
4                   None

In [3]:
import pandas as pd

# Load running order data
df_running_order = pd.read_excel('Running Order Table Pocono Cup 2025.xlsx', sheet_name='MotorSports')
xls_distance = pd.ExcelFile('Pocono Choose Line Export - Reorganized.xlsx')

# Clean running order
df_running_order = df_running_order.drop(index=0).reset_index(drop=True)
df_running_order.rename(columns={'Unnamed: 0': 'Lap'}, inplace=True)

# Define restart laps
restart_laps = [8, 36, 49, 61, 81, 88, 101, 131]
car_sheets = xls_distance.sheet_names

results = []

for lap in restart_laps:
    prev_lap = lap - 1
    if prev_lap not in df_running_order['Lap'].values:
        continue

    running_order_row = df_running_order[df_running_order['Lap'] == prev_lap].iloc[0]
    running_order = running_order_row[list(range(1, 41))].dropna().astype(int).tolist()

    restart_row = df_running_order[df_running_order['Lap'] == lap].iloc[0]
    top_16_drivers = restart_row[list(range(1, 41))].dropna().astype(int).head(16).tolist()

    for driver in top_16_drivers:
        position_before = running_order.index(driver) + 1 if driver in running_order else None

        choose_distance = None
        lane_choice = 'Outside'  # Default if missing

        sheet_name = f'Car {driver}'
        if sheet_name in car_sheets:
            df_car = pd.read_excel(xls_distance, sheet_name=sheet_name)
            if 'Restart Choose (ft)' in df_car.columns and 'Lap' in df_car.columns:
                lap_row = df_car[df_car['Lap'] == lap]
                if not lap_row.empty:
                    choose_distance = lap_row['Restart Choose (ft)'].values[0]
                    if pd.notna(choose_distance):
                        if 0 <= choose_distance <= 7:
                            lane_choice = 'Outside'
                        elif 8 <= choose_distance <= 40:
                            lane_choice = 'Inside'
                        else:
                            lane_choice = 'Outside'

        results.append({
            'Restart Lap': lap,
            'Driver (Car Number)': driver,
            'Position Before Choose': position_before,
            'Choose Distance (ft)': choose_distance,
            'Lane Choice': lane_choice
        })

# Convert to DataFrame
df_results = pd.DataFrame(results)

# Save to Excel
df_results.to_excel('Pocono_Restart_Lane_Choice.xlsx', index=False)

# Print preview
print(df_results)


     Restart Lap  Driver (Car Number)  Position Before Choose  \
0              8                   11                       1   
1              8                   17                       2   
2              8                   77                       3   
3              8                   45                       4   
4              8                   19                       5   
..           ...                  ...                     ...   
123          131                   54                      19   
124          131                   99                      10   
125          131                   20                      12   
126          131                   77                      15   
127          131                   43                      16   

    Choose Distance (ft) Lane Choice  
0                   None     Outside  
1                   None     Outside  
2                   None     Outside  
3                   None     Outside  
4                   None

In [4]:
import pandas as pd

# Load Excel data
df_running_order = pd.read_excel('Running Order Table Pocono Cup 2025.xlsx', sheet_name='MotorSports')

# Clean data
df_running_order = df_running_order.drop(index=0).reset_index(drop=True)
df_running_order.rename(columns={'Unnamed: 0': 'Lap'}, inplace=True)

# Define caution periods and restart laps
caution_periods = [(0, 7), (32, 35), (43, 48), (56, 60), (78, 80), (84, 87), (97, 100), (126, 130)]
restart_laps = [end + 1 for _, end in caution_periods]

results = []

# Loop over restart laps
for lap in restart_laps:
    prev_lap = lap - 1
    
    if prev_lap not in df_running_order['Lap'].values:
        continue

    running_order_row = df_running_order[df_running_order['Lap'] == prev_lap].iloc[0]
    running_order = running_order_row[list(range(1, 41))].dropna().astype(int).tolist()

    restart_row = df_running_order[df_running_order['Lap'] == lap].iloc[0]
    top_16_drivers = restart_row[list(range(1, 41))].dropna().astype(int).head(16).tolist()

    for driver in top_16_drivers:
        position_before = running_order.index(driver) + 1 if driver in running_order else None

        results.append({
            'Lap': lap,
            'Driver': driver,
            'Position_Before_Choose': position_before
        })

# Convert to DataFrame
results_df = pd.DataFrame(results)

# Show the DataFrame
print(results_df)


     Lap  Driver  Position_Before_Choose
0      8      11                       1
1      8      17                       2
2      8      77                       3
3      8      45                       4
4      8      19                       5
..   ...     ...                     ...
123  131      54                      19
124  131      99                      10
125  131      20                      12
126  131      77                      15
127  131      43                      16

[128 rows x 3 columns]


In [5]:
import pandas as pd

# Load Excel data
df_running_order = pd.read_excel('Running Order Table Pocono Cup 2025.xlsx', sheet_name='MotorSports')

# Clean data
df_running_order = df_running_order.drop(index=0).reset_index(drop=True)
df_running_order.rename(columns={'Unnamed: 0': 'Lap'}, inplace=True)

# Define caution periods and restart laps
caution_periods = [(0, 7), (32, 35), (43, 48), (56, 60), (78, 80), (84, 87), (97, 100), (126, 130)]
restart_laps = [end + 1 for _, end in caution_periods]

results = []

# Loop over restart laps
for lap in restart_laps:
    prev_lap = lap - 1
    
    if prev_lap not in df_running_order['Lap'].values:
        continue

    running_order_row = df_running_order[df_running_order['Lap'] == prev_lap].iloc[0]
    running_order = running_order_row[list(range(1, 41))].dropna().astype(int).tolist()

    restart_row = df_running_order[df_running_order['Lap'] == lap].iloc[0]
    top_16_drivers = restart_row[list(range(1, 41))].dropna().astype(int).head(16).tolist()

    for driver in top_16_drivers:
        position_before = running_order.index(driver) + 1 if driver in running_order else None

        results.append({
            'Lap': lap,
            'Driver': driver,
            'Position_Before_Choose': position_before
        })

# Convert to DataFrame
results_df = pd.DataFrame(results)

# Save to Excel
results_df.to_excel('Restart_Positions.xlsx', index=False)

# Print preview
print(results_df)


     Lap  Driver  Position_Before_Choose
0      8      11                       1
1      8      17                       2
2      8      77                       3
3      8      45                       4
4      8      19                       5
..   ...     ...                     ...
123  131      54                      19
124  131      99                      10
125  131      20                      12
126  131      77                      15
127  131      43                      16

[128 rows x 3 columns]
